# What is Entropy?

# Quantifying Structure

We instantiate 4 different linguistically-motivated measures of structure in the mapping between labels and representations that levereage the soft entropy estimator. While in principle these measures can be computed with respect to any labels, like probing, here we focus on labels that are already availabe for virtually any linguistic data - tokens, bigrams, and trigrams.

## Information
We have a model $f$ that maps a set of sentences $X$ to representational space $V$. For each sentence $S^k \in X$, the model takes as input a sequence of tokens $t_a^k, t_b^k, t_c^k ... \in S^k$ and returns a sequence of vectors $v_a^k, v_b^k, v_c^k ... \in V^k$ where $v_a^k$ is the vector corresponding to token $a$ when it occurs in sentence $k$. While each sequence $V^k$ is of variable length, the individual vectors are the same size. We can therefore create a list $V$ of all token representations from all sentences in the dataset

\begin{equation}
    V = [v_a^k : \forall v_a^k \in f(S^k) : \forall S^k \in X]
\end{equation}

and calculate its soft entropy. As our estimator is multi-headed the result gives us a measure of the average amount of information encoded in each subspace of the representation, $H(V)$. Given that the amount of information the model needs to encode is constant (the dataset doesn't change during training) this also tells us how compressed the model's representations are. As the soft entropy goes down, the model uses less of its attested representational space. Information is minimised (i.e. compression is maximised) as all tokens are mapped to the same vector regardless of the token and sentence they correspond to, and information is maximised when token representations are spread out uniformly across representational space. To aid interpretation we normalise this measure, as well as Variation and Regularity, so that 1.0 indicates a uniform distribution and 0.0 is one-hot. 
This measure is related to intrinsic-dimensionality, which tries to assess how much of the available representation space a model uses - however our approach is more efficient than intrinsic-dimensionality estimates using eigenvalues or k-nn clustering.



## Variation 
captures how much a property varies in representation space. Given a class of labels, like tokens, or parts of speech, it reflects whether the model learns a single global representation of each label invariant to context, or if each representation is completely unique to the sentence it occurs in. We quantify this in terms of the conditional entropy of representations, given a label, creating a list of all instances of that label $V|label$, across all contexts where it occurs

\begin{equation}
    V|label = [v_a^k \text{ if } a=label : \forall v_a^k \in V]
\end{equation}

Labels for the tokens fed into a model are virtually always known, so we can easily estimate the conditional soft entropy of $V$ given a specific token $H(V|token)$. This is minimised when all instances of a token map to the same vector regardless of the sentence they occur in, and maximised when $H(V|token) = H(V)$ indicating instances of the same token are no more likely to be similar than two tokens chosen at random. The mean variation across the set of all tokens gives us a general sense of how much the model encodes context in its internal representations.

\begin{equation}
    variation(V|set) = \frac{1}{|set|}\sum_{label}^{set}H(V|label)
\end{equation}

We can also calculate variation with respect to any features we have a set of labels for. For example, if we know the part of speech for each of the input tokens $variation(V|POS)$ could tell us if members of the same syntactic class share more information with each other than expected by chance. In the general case we just need a set of labels to condition on (e.g. part of speech, morphological case, tense etc.) when estimating $H(V|Set)$.


## Regularity
measures how structured a model's representations are with respect to a feature in the input --- in particular, whether the mapping between a label and its representation is monotonic (one-to-one).  The inverse of variation, Regularity quantifies how much knowing something about a token is going to tell us about its representation; quantifiable as the soft mutual information between a label and its representations. 

    
\begin{equation}
    regularity(Y, set) = \frac{1}{|set|}\sum^{set}_{label} H(V) - H(V|label)
\end{equation}

This is maximised when a label and its representations are monotonically aligned --- knowing the label tells us everything there is to know about the representation. As with variation we can quantify regularity with respect to individual labels in a set and mean across them to get a general notion of how aligned representations are with e.g. tokens, POS tags, or the bigrams a token is part of. This quantity is related to clustering, with higher values indicating representations for a label are more tightly clustered in space

## Disentanglement
measures how separable different labels within a set are from one another, e.g. whether separate tokens are represented in distinct regions of representational space, rather than overlapping. We measure this by assessing the Jensen-Shannon divergence between $P(V|label)$ and all other labels in the set $P(V|set \neg label)$; if tokens are distributed uniformly across a space their disentanglement will be 0, while if they are entirely separable it approaches 1.

\begin{equation}
    dis(Y, set) = JSD(P(V|label) ; P(V|set \neg label))
\end{equation}

As with previous measures we aggregate this to get an assessment of how disentangled the class of labels is. This quantity is related to cluster purity, reflecting how separable clusters for labels within a set are from each other in representational space. Our measure is related to previous assessments of entanglement but requires no pair-wise comparison of different labels. 
